# Import Libraries

In [ ]:
import mediapipe as mp
import cv2
import pandas as pd

# Initialize MediaPipe Hands

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)


# Open video file

In [ ]:
video_path = '/kaggle/input/tdl-111/Videos/Videos/hold_1.mp4'
cap = cv2.VideoCapture(video_path)

# Initialize DataFrame for storing landmarks

In [ ]:
columns = [
    "WRIST", "THUMB_CMC", "THUMB_MCP", "THUMB_IP", "THUMB_TIP",
    "INDEX_FINGER_MCP", "INDEX_FINGER_PIP", "INDEX_FINGER_DIP", "INDEX_FINGER_TIP",
    "MIDDLE_FINGER_MCP", "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP", "MIDDLE_FINGER_TIP",
    "RING_FINGER_MCP", "RING_FINGER_PIP", "RING_FINGER_DIP", "RING_FINGER_TIP",
    "PINKY_MCP", "PINKY_PIP", "PINKY_DIP", "PINKY_TIP"
]
df = pd.DataFrame(columns=columns)

# Detect Landmarks and Store in Data frame

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert BGR image to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Get hand landmarks
    results = hands.process(rgb_frame)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Check if all 21 landmarks are present
            if len(hand_landmarks.landmark) == 21:
                landmarks = [hand_landmarks.landmark[i] for i in range(21)]
                landmarks_dict = {columns[i]: [landmarks[i].x, landmarks[i].y, landmarks[i].z] for i in range(21)}
                df = pd.concat([df, pd.DataFrame(landmarks_dict)], ignore_index=True)

# Save DataFrame to CSV if data is present

In [ ]:

if not df.empty:
    df.to_csv('Hold.csv', index=False)
    print('Landmarks saved to Hold.csv')
# Release resources
cap.release()